In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import pyodbc
import pandas as pd
import time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from keras import backend as K
import datetime
K.clear_session()

%matplotlib inline

with open ('../params.json') as f:
    params = json.load(f)
    
table_prefix = params['table_prefix']
diseases = params['diseases']
case_limit = params['case_limit']
control_limit = params['control_limit']
min_enrollment = params['enrollment']
user = params['user']

chunk = True


# run_id = table_prefix_ + str(case_limit) + '_' + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
# print(run_id)

creds_file = "/home/" + user + "/creds.txt" 
creds = lines = [line.rstrip('\n') for line in open(creds_file)]

connection_string = ("Driver={ODBC Driver 17 for SQL Server};" + 
                     "server=" + creds[0] + ";" +
                     "domain=" + creds[1] + ";" +  
                     "database=" + creds[2] + ";" +
                     "uid=" + creds[3]  + ";" +
                     "pwd=" + creds[4] + ";" +
                     "ssl=require;")

cn = pyodbc.connect(connection_string, autocommit=True)
cursor = cn.cursor()

directory = '../../data/diseaes_replaced' + str(table_prefix) + '_' + str(case_limit)
output_dir = '../../outputs/' + str(table_prefix) + _ + str(case_limit) + '_' +\
             'gd'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def get_data(path):
    ''' Returns dataframe with columns: 'path', 'word'.'''
    datadir = Path(path)
    files = [(str(f), f.parts[-1]) for f in datadir.glob('*.csv.gz') if f]
    df = pd.DataFrame(files, columns=['path', 'word'])
    
    return df

/home/anaconda/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
back_window = 25
train = get_data(directory)
X = train.path
print(X)
X.to_csv(directory + '/files_df.csv')

0       ../../data/diseaes_replacedtest_10000/seq_1842...
1       ../../data/diseaes_replacedtest_10000/seq_1734...
2       ../../data/diseaes_replacedtest_10000/seq_1151...
3       ../../data/diseaes_replacedtest_10000/seq_1865...
4       ../../data/diseaes_replacedtest_10000/seq_1788...
5       ../../data/diseaes_replacedtest_10000/seq_8770...
6       ../../data/diseaes_replacedtest_10000/seq_8740...
7       ../../data/diseaes_replacedtest_10000/seq_1540...
8       ../../data/diseaes_replacedtest_10000/seq_8500...
9       ../../data/diseaes_replacedtest_10000/seq_1131...
10      ../../data/diseaes_replacedtest_10000/seq_3560...
11      ../../data/diseaes_replacedtest_10000/seq_1888...
12      ../../data/diseaes_replacedtest_10000/seq_1666...
13      ../../data/diseaes_replacedtest_10000/seq_5430...
14      ../../data/diseaes_replacedtest_10000/seq_1964...
15      ../../data/diseaes_replacedtest_10000/seq_7350...
16      ../../data/diseaes_replacedtest_10000/seq_1196...
17      ../../

In [3]:
from keras.utils import to_categorical
import pickle as pkl

def get_seqs(path, vocab_size):
    #print(path)
    seq = pd.read_csv(path)
    #print(seq.shape)
    
    X = []
    decoder_target = []
    y_time = []
    
    for index, row in seq.iterrows():
        if row['fromIndex'] >= 0 and index < seq.shape[0] - 1:
            X.append(to_categorical(seq.iloc[index+1-back_window:index+1]['InputCode'].values, 
                                    num_classes=vocab_size))
            decoder_target.append(to_categorical(seq.iloc[index+2-back_window:index+2]['InputCode'].values, 
                                 num_classes=vocab_size))
            y_time.append(seq.iloc[index+1]['fromIndex'])
    
    
    # encoder input, decoder input, decoder target, y_time
    return (np.asarray(X), np.asarray(X),
            np.asarray(decoder_target), np.asarray(y_time))

    
def batch_generator(X, batch_size=32):
    vocab_dict = pkl.load(open(directory + '/input_token_index.pkl', 'rb'))
    vocab_size = len(vocab_dict)
    while True:
        # choose batch_size random images / labels from the data
        idx = np.random.randint(0, X.shape[0], batch_size)
        x_file = X[idx]
        
        if str(x_file.values[0]) == 'nan':
            continue
        encoder_input, decoder_input, decoder_target, y_time = get_seqs(str(x_file.values[0]), len(vocab_dict))
        
        for i in range(encoder_input.shape[0] / batch_size):
            if (encoder_input.ndim == 3) and (((i+1)*batch_size)-(i*batch_size)) > 1:
                yield ([encoder_input[(i*batch_size):(i+1)*batch_size],
                        decoder_input[(i*batch_size):(i+1)*batch_size]], 
                       [decoder_target[(i*batch_size):(i+1)*batch_size],
                        y_time[(i*batch_size):(i+1)*batch_size,]])
            else:
                pass 
#                 print(encoder_input.shape, decoder_input.shape, decoder_target.shape, y_time.shape)
#                 print(X[idx])
#                 raise Exception('stop')

In [4]:
# print(get_seqs('../../data/test_1000/seq_1200_1300.csv.gz'))
%time [ei_batch, di_batch], [dt_batch, y_time_batch] = batch_generator(X).next()
print('shapes: ', ei_batch.shape, di_batch.shape, dt_batch.shape, y_time_batch.shape)

CPU times: user 9.17 s, sys: 3.12 s, total: 12.3 s
Wall time: 12.4 s
('shapes: ', (32, 25, 1436), (32, 25, 1436), (32, 25, 1436), (32,))


In [5]:
from keras.models import Model
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Lambda, GRU, Embedding, GaussianNoise
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
from keras_exp.multigpu import print_mgpu_modelsummary

def get_model(num_tokens, latent_dim):
    encoder_inputs = Input(shape=(None, num_tokens), name='encoder_input')
    encoder = GRU(latent_dim, return_state=True, name='encoded')
    encoder_outputs, state_h = encoder(encoder_inputs)
                                       
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, num_tokens), name='decoder_input')
    noisy_inputs = GaussianNoise(0.2)(decoder_inputs)
    decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _ = decoder_gru(noisy_inputs, initial_state=state_h)
    decoder_dense = Dense(num_tokens, activation='softmax', name='ae')
    decoder_outputs = decoder_dense(decoder_outputs)

    time_internal = Dense(100, activation='relu')(encoder_outputs)
    time_dense = Dense(1, activation='relu', name='time')
    time_output = time_dense(time_internal)

    model = Model([encoder_inputs, decoder_inputs], [decoder_outputs, time_output])

    model.summary()
    return model

NCCL support available


In [6]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

print(tf.__version__)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
K.set_session(sess)  # set this TensorFlow session as the default session for Keras

print(type(X), X.shape[0]*0.8)
Xt = X[int(X.shape[0]*0.8):]
X = X[:int(X.shape[0]*0.8)]

Xt = Xt.reset_index(drop=True)

print(len(X), len(Xt)) 

fixed_length = 25
fixed_vocab = len(pkl.load(open(directory + '/input_token_index.pkl', 'rb')))
# epochs = 100
latent_dim = 10

model = get_model(fixed_vocab, latent_dim)

losses = {'ae':'categorical_crossentropy', 'time':'mse'}
loss_weights = {'ae':1, 'time':2}
earlystopper = EarlyStopping(monitor='loss', patience=100, verbose=1)
tbCallback = TensorBoard(log_dir='../../logs/gd', write_graph=True)

filepath = output_dir + "/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='min')

model.compile(optimizer='adam', loss=losses, loss_weights=loss_weights)
model.save(output_dir + '/empty_model.model')

1.8.0
(<class 'pandas.core.series.Series'>, 1600.0)
(1600, 400)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None, 1436)   0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      (None, None, 1436)   0                                            
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, None, 1436)   0           decoder_input[0][0]              
__________________________________________________________________________________________________
encoded (GRU)                   [(None, 10), (None,  43410       encoder_input[0][0]              
_____________________________________________

/home/anaconda/anaconda3/envs/py27/lib/python2.7/site-packages/keras/engine/network.py:888: UserWarning: Layer gru_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoded/while/Exit_2:0' shape=(?, 10) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [7]:
batch_size = 1024
train_gen = batch_generator(X, batch_size=batch_size)
valid_gen = batch_generator(Xt, batch_size=batch_size)
print(len(X), len(Xt))

(1600, 400)


In [8]:
## import time
start_time = time.time()
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    validation_steps=10,
    epochs=2000,
    steps_per_epoch=100, #X.shape[0], - we do smaller steps per epoch to see training progress
    use_multiprocessing = True,
    workers = 8,
    callbacks=[earlystopper, checkpoint, tbCallback])
print(time.time()-start_time)

model.save(output_dir + '/trained_model_weights.model')

/home/anaconda/anaconda3/envs/py27/lib/python2.7/site-packages/keras/engine/training_generator.py:44: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/2000
 99/100 [============================>.] - ETA: 1s - loss: 1627532.2192 - ae_loss: 7.1540 - time_loss: 813762.5333

/home/anaconda/anaconda3/envs/py27/lib/python2.7/site-packages/keras/engine/training_generator.py:272: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


100/100 [==============================] - 194s 2s/step - loss: 1619760.7477 - ae_loss: 7.1497 - time_loss: 809876.7998 - val_loss: 1379612.7063 - val_ae_loss: 6.8105 - val_time_loss: 689802.9437

Epoch 00001: saving model to ../../outputs/test10000_gd/weights.01-1379612.71.hdf5
Epoch 2/2000
100/100 [==============================] - 175s 2s/step - loss: 1538795.3855 - ae_loss: 6.5377 - time_loss: 769394.4263 - val_loss: 1348884.8313 - val_ae_loss: 6.4467 - val_time_loss: 674439.1930

Epoch 00002: saving model to ../../outputs/test10000_gd/weights.02-1348884.83.hdf5
Epoch 3/2000
100/100 [==============================] - 176s 2s/step - loss: 1401357.9162 - ae_loss: 6.4416 - time_loss: 700675.7404 - val_loss: 890498.7719 - val_ae_loss: 6.4131 - val_time_loss: 445246.1789

Epoch 00003: saving model to ../../outputs/test10000_gd/weights.03-890498.77.hdf5
Epoch 4/2000
100/100 [==============================] - 177s 2s/step - loss: 1384201.6782 - ae_loss: 6.4186 - time_loss: 692097.6289 - v

100/100 [==============================] - 162s 2s/step - loss: 514920.6326 - ae_loss: 6.2826 - time_loss: 257457.1754 - val_loss: 609217.3953 - val_ae_loss: 6.1834 - val_time_loss: 304605.6078

Epoch 00057: saving model to ../../outputs/test10000_gd/weights.57-609217.40.hdf5
Epoch 58/2000
100/100 [==============================] - 157s 2s/step - loss: 522173.1850 - ae_loss: 6.2377 - time_loss: 261083.4739 - val_loss: 470564.6359 - val_ae_loss: 6.2347 - val_time_loss: 235279.2008

Epoch 00058: saving model to ../../outputs/test10000_gd/weights.58-470564.64.hdf5
Epoch 59/2000
100/100 [==============================] - 159s 2s/step - loss: 531667.5684 - ae_loss: 6.2663 - time_loss: 265830.6506 - val_loss: 650148.4688 - val_ae_loss: 6.3580 - val_time_loss: 325071.0609

Epoch 00059: saving model to ../../outputs/test10000_gd/weights.59-650148.47.hdf5
Epoch 60/2000
100/100 [==============================] - 158s 2s/step - loss: 539380.5681 - ae_loss: 6.2914 - time_loss: 269687.1386 - val_lo

100/100 [==============================] - 153s 2s/step - loss: 439043.4677 - ae_loss: 6.0255 - time_loss: 219518.7198 - val_loss: 470264.7469 - val_ae_loss: 5.9736 - val_time_loss: 235129.3844

Epoch 00113: saving model to ../../outputs/test10000_gd/weights.113-470264.75.hdf5
Epoch 114/2000
100/100 [==============================] - 155s 2s/step - loss: 577380.7698 - ae_loss: 6.0507 - time_loss: 288687.3597 - val_loss: 436523.4000 - val_ae_loss: 6.0881 - val_time_loss: 218258.6562

Epoch 00114: saving model to ../../outputs/test10000_gd/weights.114-436523.40.hdf5
Epoch 115/2000
100/100 [==============================] - 153s 2s/step - loss: 506308.1648 - ae_loss: 6.0969 - time_loss: 253151.0354 - val_loss: 539267.4750 - val_ae_loss: 6.0801 - val_time_loss: 269630.6984

Epoch 00115: saving model to ../../outputs/test10000_gd/weights.115-539267.47.hdf5
Epoch 116/2000
100/100 [==============================] - 154s 2s/step - loss: 518750.4729 - ae_loss: 6.0192 - time_loss: 259372.2286 - 

100/100 [==============================] - 150s 2s/step - loss: 502348.4740 - ae_loss: 5.9858 - time_loss: 251171.2450 - val_loss: 430953.2641 - val_ae_loss: 5.8163 - val_time_loss: 215473.7266

Epoch 00194: saving model to ../../outputs/test10000_gd/weights.194-430953.26.hdf5
Epoch 195/2000
100/100 [==============================] - 156s 2s/step - loss: 493524.5300 - ae_loss: 5.9639 - time_loss: 246759.2839 - val_loss: 557420.6687 - val_ae_loss: 5.9186 - val_time_loss: 278707.3703

Epoch 00195: saving model to ../../outputs/test10000_gd/weights.195-557420.67.hdf5
Epoch 196/2000
100/100 [==============================] - 149s 1s/step - loss: 507062.3654 - ae_loss: 5.9008 - time_loss: 253528.2330 - val_loss: 507968.6219 - val_ae_loss: 5.9106 - val_time_loss: 253981.3570

Epoch 00196: saving model to ../../outputs/test10000_gd/weights.196-507968.62.hdf5
Epoch 197/2000
100/100 [==============================] - 150s 2s/step - loss: 449541.1234 - ae_loss: 5.9519 - time_loss: 224767.5861 - 

100/100 [==============================] - 159s 2s/step - loss: 491762.9982 - ae_loss: 5.8830 - time_loss: 245878.5584 - val_loss: 357360.0297 - val_ae_loss: 5.7467 - val_time_loss: 178677.1414

Epoch 00222: saving model to ../../outputs/test10000_gd/weights.222-357360.03.hdf5
Epoch 223/2000
100/100 [==============================] - 158s 2s/step - loss: 538079.3893 - ae_loss: 5.9761 - time_loss: 269036.7079 - val_loss: 367672.6227 - val_ae_loss: 5.7093 - val_time_loss: 183833.4570

Epoch 00223: saving model to ../../outputs/test10000_gd/weights.223-367672.62.hdf5
Epoch 224/2000
100/100 [==============================] - 154s 2s/step - loss: 504074.2978 - ae_loss: 5.9476 - time_loss: 252034.1759 - val_loss: 501397.5312 - val_ae_loss: 5.9189 - val_time_loss: 250695.8062

Epoch 00224: saving model to ../../outputs/test10000_gd/weights.224-501397.53.hdf5
Epoch 225/2000
100/100 [==============================] - 153s 2s/step - loss: 466134.9270 - ae_loss: 5.9156 - time_loss: 233064.5054 - 